In [ ]:
#!/usr/bin/env python3
"""
Proteus-compatible Python code for driving an 8-dot Braille cell
(using a 74HC595 shift register and a ULN2803).

This version displays a single character (hardcoded in main()) that
remains displayed indefinitely.
"""

import RPi.GPIO as GPIO
import time

# -------------------------------------------------
# GPIO PIN ASSIGNMENTS (using BCM numbering)
# -------------------------------------------------
DATA_PIN  = 17  # SER on 74HC595
CLOCK_PIN = 24  # SRCLK on 74HC595
LATCH_PIN = 22  # RCLK on 74HC595

# -------------------------------------------------
# BRAILLE LOOKUP TABLE
# -------------------------------------------------
# Bits: [dot8 dot7 dot6 dot5 dot4 dot3 dot2 dot1]
# Standard 6-dot Braille for letters a–z.
# For digits, we use the pattern of letters a–j with dot7 (bit6) set.
braille_dict = {
    # Letters a-z (default: no dot7 or dot8)
    'a': 0b00000001,  # dot1
    'b': 0b00000011,  # dots1,2
    'c': 0b00001001,  # dots1,4
    'd': 0b00011001,  # dots1,4,5
    'e': 0b00010001,  # dots1,5
    'f': 0b00001011,  # dots1,2,4
    'g': 0b00011011,  # dots1,2,4,5
    'h': 0b00010011,  # dots1,2,5
    'i': 0b00001010,  # dots2,4
    'j': 0b00011010,  # dots2,4,5
    'k': 0b00000101,  # dots1,3
    'l': 0b00000111,  # dots1,2,3
    'm': 0b00001101,  # dots1,3,4
    'n': 0b00011101,  # dots1,3,4,5
    'o': 0b00010101,  # dots1,3,5
    'p': 0b00001111,  # dots1,2,3,4
    'q': 0b00011111,  # dots1,2,3,4,5
    'r': 0b00010111,  # dots1,2,3,5
    's': 0b00001110,  # dots2,3,4
    't': 0b00011110,  # dots2,3,4,5
    'u': 0b00100101,  # dots1,3,6
    'v': 0b00100111,  # dots1,2,3,6
    'w': 0b00111010,  # dots2,4,5,6
    'x': 0b00101101,  # dots1,3,4,6
    'y': 0b00111101,  # dots1,3,4,5,6
    'z': 0b00110101,  # dots1,3,5,6

    # Digits 1-9,0 = letters a-j with dot7 (bit6) set
    '1': 0b01000001,  # 'a' + dot7
    '2': 0b01000011,  # 'b' + dot7
    '3': 0b01001001,  # 'c' + dot7
    '4': 0b01011001,  # 'd' + dot7
    '5': 0b01010001,  # 'e' + dot7
    '6': 0b01001011,  # 'f' + dot7
    '7': 0b01011011,  # 'g' + dot7
    '8': 0b01010011,  # 'h' + dot7
    '9': 0b01001010,  # 'i' + dot7
    '0': 0b01011010,  # 'j' + dot7
}

# -------------------------------------------------
# SETUP FUNCTION
# -------------------------------------------------
def setup():
    GPIO.setmode(GPIO.BCM)
    GPIO.setup(DATA_PIN,  GPIO.OUT)
    GPIO.setup(CLOCK_PIN, GPIO.OUT)
    GPIO.setup(LATCH_PIN, GPIO.OUT)

    GPIO.output(DATA_PIN,  GPIO.LOW)
    GPIO.output(CLOCK_PIN, GPIO.LOW)
    GPIO.output(LATCH_PIN, GPIO.LOW)

# -------------------------------------------------
# SHIFT OUT 8 BITS (MSB FIRST)
# -------------------------------------------------
def shift_out_8_bits(data_byte):
    """
    Bit-bang 8 bits (MSB first) to the 74HC595.
    """
    for bit_index in range(8):
        bit_to_send = (data_byte & (1 << (7 - bit_index))) >> (7 - bit_index)
        GPIO.output(DATA_PIN, bit_to_send)
        GPIO.output(CLOCK_PIN, GPIO.HIGH)
        time.sleep(0.000002)
        GPIO.output(CLOCK_PIN, GPIO.LOW)

def latch_output():
    """
    Latch the shifted data to the 74HC595 output register.
    """
    GPIO.output(LATCH_PIN, GPIO.HIGH)
    time.sleep(0.000002)
    GPIO.output(LATCH_PIN, GPIO.LOW)

# -------------------------------------------------
# DISPLAY A SINGLE CHARACTER
# -------------------------------------------------
def display_character(ch, write_mode=False):
    """
    Looks up the Braille pattern for character 'ch' (a-z or 0-9).
    If write_mode is True, dot8 (bit7) is activated.
    """
    pattern = braille_dict.get(ch.lower(), 0)
    if write_mode:
        pattern |= (1 << 7)  # Activate dot8 (bit7)
    shift_out_8_bits(pattern)
    latch_output()

# -------------------------------------------------
# MAIN (Proteus Entry Point)
# -------------------------------------------------
def main():
    """
    Proteus calls this function automatically.
    This version displays a single character and then holds the display.
    """
    setup()
    # Define the single character and write mode you want to display.
    character_to_display = '2'
    write_mode = False  # Change to True to activate dot8
    
    # Display the character once.
    display_character(character_to_display, write_mode)
    print("Displaying '{}' with write_mode={}".format(character_to_display, write_mode))
    
    # Hold the display indefinitely.
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        pass
    finally:
        GPIO.cleanup()
        print("GPIO cleaned up. Simulation ended.")